# Sample workflow: 2000 block group parts to 2010 tracts

## Starting from a subset of 2010 Delaware blocks

For further background information see:
* **Schroeder, J. P**. 2007. *Target-density weighting interpolation and uncertainty evaluation for temporal analysis of census data*. Geographical Analysis 39 (3):311–335.

#### NHGIS [block crosswalks](https://www.nhgis.org/user-resources/geographic-crosswalks)

In [1]:
%load_ext watermark
%watermark

2020-05-16T12:43:17-04:00

CPython 3.7.6
IPython 7.13.0

compiler   : Clang 9.0.1 
system     : Darwin
release    : 19.4.0
machine    : x86_64
processor  : i386
CPU cores  : 4
interpreter: 64bit


In [2]:
import nhgisxwalk
import inspect
import pandas

%load_ext autoreload
%autoreload 2
%watermark -w
%watermark -iv

watermark 2.0.2
nhgisxwalk 0.0.2
pandas     1.0.3



### Source and target years for the crosswalk

In [3]:
source_year, target_year = "2000", "2010"

### Source-target building base

In [4]:
subset_data_dir = "../testing_data_subsets"
base_xwalk_name = "/nhgis_blk%s_blk%s_gj.csv.zip" % (source_year, target_year)
base_xwalk_file = subset_data_dir + base_xwalk_name
data_types = nhgisxwalk.str_types(["GJOIN%s"%source_year, "GJOIN%s"%target_year])
base_xwalk = pandas.read_csv(base_xwalk_file, index_col=0, dtype=data_types)
base_xwalk.head()

,GJOIN2000,GJOIN2010,WEIGHT,PAREA
0,G10000100401001000,G10000100401001000,1.000000,1.000000
1,G10000100401001001,G10000100401001001,0.999981,0.999988
2,G10000100401001001,G10000100401001003,0.000019,0.000012
3,G10000100401001002,G10000100401001002,1.000000,1.000000
4,G10000100401001003,G10000100401001003,1.000000,1.000000


### Convenience code shorthand/lookup

In [5]:
print(inspect.getsource(nhgisxwalk.valid_geo_shorthand))

def valid_geo_shorthand(shorthand_name=True):
    """Shorthand lookups for census geographies."""
    lookup = {
        "blk": "block",
        "bgp": "block group part",
        "bkg": "block group",
        "trt": "tract",
        "cty": "county",
    }
    if not shorthand_name:
        lookup = {v: k for k, v in lookup.items()}
    return lookup



In [6]:
nhgisxwalk.valid_geo_shorthand(shorthand_name=False)

{'block': 'blk',
 'block group part': 'bgp',
 'block group': 'bkg',
 'tract': 'trt',
 'county': 'cty'}

### Instantiate an `nhgisxwalk.GeoCrossWalk` object
##### see [nhgisxwalk.GeoCrossWalk](https://github.com/jGaboardi/nhgisxwalk/blob/92b4fe55de0a9c53d0315dcda8ec121faaf20aef/nhgisxwalk/geocrosswalk.py#L19) for full details

In [7]:
nhgisxwalk.desc_code_2000_SF1b

{'Persons': {'Persons': 'Universe',
  'NP001A': 'Source code',
  'FXS': 'NHGIS code',
  'Total': 'FXS001'},
 'Families': {'Families': 'Universe',
  'NP031A': 'Source code',
  'F2V': 'NHGIS code',
  'Total': 'F2V001'},
 'Households': {'Households': 'Universe',
  'NP010A': 'Source code',
  'FY4': 'NHGIS code',
  'Total': 'FY4001'},
 'Housing Units': {'Housing Units': 'Universe',
  'NH001A': 'Source code',
  'FV5': 'NHGIS code',
  'Total': 'FV5001'}}

In [8]:
input_vars = [
    nhgisxwalk.desc_code_2000_SF1b["Persons"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Families"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Households"]["Total"],
    nhgisxwalk.desc_code_2000_SF1b["Housing Units"]["Total"]
]
input_vars

['FXS001', 'F2V001', 'FY4001', 'FV5001']

In [9]:
input_var_tags = ["pop", "fam", "hh", "hu"]

In [10]:
subset_state = "10"
bgp2000_to_trt2010 = nhgisxwalk.GeoCrossWalk(
    base_xwalk,
    source_year=source_year,
    target_year=target_year,
    source_geo="bgp",
    target_geo="trt",
    base_source_table=subset_data_dir+"/2000_block.csv.zip",
    input_var=input_vars,
    weight_var=input_var_tags,
    stfips=subset_state
)
bgp2000_to_trt2010.xwalk[1020:1031]

,bgp2000,trt2010,wt_pop,wt_fam,wt_hh,wt_hu
1020,G10000509355299999051301U2,G1000050051301,1.000000,1.000000,1.000000,1.000000
1021,G10000509355299999051302R1,G1000050051302,1.000000,1.000000,1.000000,1.000000
1022,G10000509355299999051302R2,G1000050051302,1.000000,1.000000,1.000000,1.000000
1023,G10000509355299999051302U1,G1000050051302,1.000000,1.000000,1.000000,1.000000
1024,G10000509355299999051303R1,G1000050051303,1.000000,1.000000,1.000000,1.000000
1025,G10000509355299999051303U1,G1000050051303,1.000000,1.000000,1.000000,1.000000
1026,G10000509355299999051304R1,G1000050051305,0.680605,0.633909,0.657366,0.659502
1027,G10000509355299999051304R1,G1000050051306,0.319167,0.365782,0.342282,0.340111
1028,G10000509355299999051304R1,G1000050051400,0.000227,0.000309,0.000352,0.000387
1029,G10000509355299999051304R2,G1000050051305,0.802661,0.817568,0.820896,0.836237


### Write crosswalk to a `.csv`

In [11]:
res_dir = "../../results/"
bgp2000_to_trt2010.xwalk_to_csv(loc=res_dir)

### Read crosswalk from a `.csv`

In [12]:
fname = res_dir + "bgp2000_to_trt2010_%s" % subset_state
bgp2000_to_trt2010 = nhgisxwalk.GeoCrossWalk.xwalk_from_csv(fname)
bgp2000_to_trt2010[1020:1031]

,bgp2000,trt2010,wt_pop,wt_fam,wt_hh,wt_hu
1020,G10000509355299999051301U2,G1000050051301,1.000000,1.000000,1.000000,1.000000
1021,G10000509355299999051302R1,G1000050051302,1.000000,1.000000,1.000000,1.000000
1022,G10000509355299999051302R2,G1000050051302,1.000000,1.000000,1.000000,1.000000
1023,G10000509355299999051302U1,G1000050051302,1.000000,1.000000,1.000000,1.000000
1024,G10000509355299999051303R1,G1000050051303,1.000000,1.000000,1.000000,1.000000
1025,G10000509355299999051303U1,G1000050051303,1.000000,1.000000,1.000000,1.000000
1026,G10000509355299999051304R1,G1000050051305,0.680605,0.633909,0.657366,0.659502
1027,G10000509355299999051304R1,G1000050051306,0.319167,0.365782,0.342282,0.340111
1028,G10000509355299999051304R1,G1000050051400,0.000227,0.000309,0.000352,0.000387
1029,G10000509355299999051304R2,G1000050051305,0.802661,0.817568,0.820896,0.836237


-----------------------------------------------